# Log viewer

In [ ]:
# !pip install pandas plotly dash dash_core_components dash_html_components

In [ ]:
import re, textwrap
import pandas as pd
import plotly.graph_objects as go

In [ ]:
log_filename = './test/test.log'
with open(log_filename, 'r', encoding='utf-8-sig') as f:
    log_string = f.read()

In [ ]:
assert len(log_string)

In [ ]:
# Define your regex pattern
pattern = (
    r"(?P<log_level>INFO|DEV|DEBUG|ERROR|FATAL)"
    r" \| "
    r"(?P<timestamp>\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d+[+-]\d+)"
    r" \| "
    r"(?P<name>exe: \w+)"
    r" \| "
    r"(?P<process_id>pid:\d+)"
    r" \| "
    r"(?P<thread_id>tid:\d+)"
    r" \| "
    r"(?P<message>.*?)(?=(?:INFO|DEV|DEBUG|ERROR|FATAL) \| |\Z)"
)

In [ ]:
# Initialize start position
start = 0

# Find all matches
for match in re.finditer(pattern, log_string, re.DOTALL):
    # Get the start and end of the current match
    match_start, match_end = match.span()

    # If there's a gap between the last match and this one
    if start != match_start:
        # Print the unmatched part
        print(log_string[start:match_start])

    # Update start position
    start = match_end

# Print remaining unmatched part, if any
if start < len(log_string):
    print(log_string[start:])

assert start == len(log_string)

In [ ]:
matches = re.findall(pattern, log_string, re.DOTALL)
df = pd.DataFrame(matches, columns=['log_level', 'timestamp', 'name', 'process_id', 'thread_id', 'message'])

In [ ]:
df['message'] = df['message'].str.strip()  # Strip leading/trailing white space from messages

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])  # Convert timestamp to datetime

In [ ]:
df=df.sort_values('timestamp') # Sort by timestamp, not expected to change order of messages

In [ ]:
# Print the number of messages per thread_id
print(f'df num of rows = {df.shape[0]} should match total messages {sum(df["thread_id"].value_counts())}')

df['thread_id'].value_counts()

In [ ]:
# df.style.set_properties(**{'text-align': 'left'})

In [ ]:
# prepare hover text, 
# message can be very long, have newlines, spaces

# first replace all newlines with <br>
df['hover_text'] = df['message'].str.replace('\n', '<br>')

In [ ]:
import re

def insert_br(text, line_length):
    # Split the text into lines at existing <br> tags
    lines = re.split('<br>', text)

    # For each line, insert <br> tags every n characters, respecting words
    lines = ['<br>'.join(textwrap.wrap(line, line_length)) for line in lines]

    # Join the lines back together with <br> tags
    text = '<br>'.join(lines)

    return text

# Apply the function to your DataFrame column
df['hover_text'] = df['hover_text'].apply(lambda x: insert_br(x, 80))  # Change 50 to your preferred line length


In [ ]:
# at the end replace all spaces with &nbsp;
df['hover_text'] = df['hover_text'].str.replace(' ', '&nbsp;')

In [ ]:
df.head()

In [ ]:
# Map log_level to marker colors
color_marker = {
    'INFO': 'blue', 
    'DEV': 'lightgreen', 
    'DEBUG': 'orange', 
    'ERROR': 'red', 
    'FATAL': 'red'
}

color_line = {
    'INFO': 'DarkSlateGrey', 
    'DEV': 'DarkSlateGrey', 
    'DEBUG': 'DarkSlateGrey', 
    'ERROR': 'yellow', 
    'FATAL': 'maroon'
}

# Map log_level to marker size
size_map = {
    'INFO': 10, 
    'DEV': 8, 
    'DEBUG': 8, 
    'ERROR': 12, 
    'FATAL': 15
}

In [ ]:
def get_figure(input_df):
    fig=go.Figure()

    # Add a trace for each log_level
    for log_level in input_df['log_level'].unique():
        df_log_level = input_df[input_df['log_level'] == log_level]
        fig.add_trace(
            go.Scatter(
                x=df_log_level['thread_id'], 
                y=df_log_level.index, 

                text=df_log_level['hover_text'],
                mode='markers', 
                marker=dict(
                    color=color_marker[log_level], 
                    symbol='diamond', 
                    size=size_map[log_level],
                    line=dict(
                        width=1, 
                        color=color_line[log_level]
                    )
                ), 
                name=log_level
            )
        )

    # Update layout to show text on hover
    fig.update_layout(
        hovermode='closest', title=f'Log:{log_filename}', 
            yaxis=dict(
            autorange='reversed',
        ),
        hoverlabel=dict(
            font=dict(
                size=20,  # Set font size of hover text
            )
        ),
        width=800,  # Set width of plot in pixels
        height=1000,  # Set height of plot in pixels
    )
    return fig

# Show the plot
# get_figure(df).show()

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Input(id='filter-field', type='text', placeholder='Filter', style={'height': 20, 'width': '50%'}),
    dcc.Graph(id='log-graph')
], style={
    'width': '100%',
    'height': '100%',})

@app.callback(
    dash.dependencies.Output('log-graph', 'figure'),
    [dash.dependencies.Input('filter-field', 'value')]
)
def update_graph(filter_string):
    if filter_string is None or filter_string == '':
        df_filtered = df
    else:
        df_filtered = df[df['message'].str.contains(filter_string, case=False)]
    # print(f'filter_string={filter_string}')

    return get_figure(df_filtered)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)